# nnsigh walkthrough
https://nnsight.net/notebooks/tutorials/walkthrough/

In [1]:
from collections import OrderedDict
import torch

import nnsight
from nnsight import NNsight

In [11]:
!pip show nnsight

Name: nnsight
Version: 0.2.21
Summary: Package for interpreting and manipulating the internals of deep learning models.
Home-page: https://github.com/ndif-team/nnsight
Author: 
Author-email: Jaden Fiotto-Kaufman <jadenfk@outlook.com>
License: 
Location: /home/ailab/python_venv/feature_circuits/lib/python3.11/site-packages
Requires: accelerate, diffusers, einops, protobuf, pydantic, python-socketio, sentencepiece, tokenizers, torch, torchvision, transformers
Required-by: 


## Tracing Context
Everything within the tracing context operates on the intervention graph.

In [2]:
input_size = 5
hidden_dims = 10
output_size = 2

net = torch.nn.Sequential(
    OrderedDict(
        [
            ("layer1", torch.nn.Linear(input_size, hidden_dims)),
            ("layer2", torch.nn.Linear(hidden_dims, output_size)),
        ]
    )
).requires_grad_(False)

In [3]:
tiny_model = NNsight(net)
tiny_model

Sequential(
  (layer1): Linear(in_features=5, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=2, bias=True)
)

In [4]:
# random input
input = torch.rand((1, input_size))

with tiny_model.trace(input) as tracer:
    pass

## Getting

In [5]:
#output of the model as a whole
with tiny_model.trace(input) as tracer:

    output = tiny_model.output.save()

print(output)

tensor([[0.5712, 0.0162]])


In [6]:
#Let’s access the output of the first layer
with tiny_model.trace(input) as tracer:

    l1_output = tiny_model.layer1.output.save()

print(l1_output)


tensor([[ 0.2227,  0.2501,  0.1167,  0.1502, -0.1809, -0.3574,  0.5037,  0.4595,
          0.2806, -0.3728]])


In [7]:
with tiny_model.trace(input):

    l2_input = tiny_model.layer2.input.save()

print(l2_input)

((tensor([[ 0.2227,  0.2501,  0.1167,  0.1502, -0.1809, -0.3574,  0.5037,  0.4595,
          0.2806, -0.3728]]),), {})


## Functions, Methods, and Operations

In [8]:
with tiny_model.trace(input):

    # Note we don't need to call .save() on the output,
    # as we're only using its value within the tracing context.
    l1_output = tiny_model.layer1.output

    # We do need to save the argmax tensor however,
    # as we're using it outside the tracing context.
    l1_amax = torch.argmax(l1_output, dim=1).save()

print(l1_amax[0])

tensor(6)


In [9]:
"""Run the model with the given input. 
When the output of tiny_model.layer1 is computed, take its sum. 
Then do the same for tiny_model.layer2. 
Now that both of those are computed, add them and make sure 
not to delete this value as I wish to use it outside of the 
tracing context."""
with tiny_model.trace(input):

    value = (tiny_model.layer1.output.sum() + tiny_model.layer2.output.sum()).save()

print(value)

tensor(1.6600)


## Custom Functions

In [10]:
# Take a tensor and return the sum of its elements
def tensor_sum(tensor):
    flat = tensor.flatten()
    total = 0
    for element in flat:
        total += element.item()

    return torch.tensor(total)

with tiny_model.trace(input) as tracer:

    # Specify the function name and its arguments (in a comma-separated form) to add to the intervention graph
    custom_sum = nnsight.apply(tensor_sum, tiny_model.layer1.output).save()
    sum = tiny_model.layer1.output.sum()
    sum.save()


print(custom_sum, sum)

AttributeError: module 'nnsight' has no attribute 'apply'

## Setting

In [ ]:
# let’s set the first dimension of the first layer’s output to 0. 
# NNsight makes this really easy using the ‘=’ operator

with tiny_model.trace(input):

    # Save the output before the edit to compare.
    # Notice we apply .clone() before saving as the setting operation is in-place.
    l1_output_before = tiny_model.layer1.output.clone().save()

    # Access the 0th index of the hidden state dimension and set it to 0.
    tiny_model.layer1.output[:, 0] = 0

    # Save the output after to see our edit.
    l1_output_after = tiny_model.layer1.output.save()

print("Before:", l1_output_before)
print("After:", l1_output_after)

Before: tensor([[-0.5135, -0.2158, -0.3256, -0.1693,  0.0873,  0.7608, -0.5865,  0.5870,
         -0.8724, -0.0315]])
After: tensor([[ 0.0000, -0.2158, -0.3256, -0.1693,  0.0873,  0.7608, -0.5865,  0.5870,
         -0.8724, -0.0315]])


In [12]:
with tiny_model.trace(input):

    # Save the output before the edit to compare.
    # Notice we apply .clone() before saving as the setting operation is in-place.
    l1_output_before = tiny_model.layer1.output.clone().save()

    # Access the last index of the hidden state dimension and set it to 0.
    tiny_model.layer1.output[:, hidden_dims -1] = 0

    # Save the output after to see our edit.
    l1_output_after = tiny_model.layer1.output.save()

print("Before:", l1_output_before)
print("After:", l1_output_after)

Before: tensor([[ 0.2227,  0.2501,  0.1167,  0.1502, -0.1809, -0.3574,  0.5037,  0.4595,
          0.2806, -0.3728]])
After: tensor([[ 0.2227,  0.2501,  0.1167,  0.1502, -0.1809, -0.3574,  0.5037,  0.4595,
          0.2806,  0.0000]])


# Scan and Validate